In [2]:
"""
1000 cpu s / trial / GRB
* 1000 trials for 2268 GRBs
= 630 kcpu hr
= 400jobs * 16cpus * 98 hr
"""

'\n1000 cpu s / trial / GRB\n* 1000 trials for 2268 GRBs\n= 630 kcpu hr\n= 400jobs * 16cpus * 98 hr\n'

In [1]:
import sys
import os
import glob
import numpy as np
import pandas as pd
sys.path.append('../../../')
from greco_grb.scripts.utils import *

In [2]:
from greco_grb.scripts import SETTING
paths = SETTING.PATH(osg=False)
print(paths)
LOCATION = paths.LOCATION
USER = paths.USER
ICDATA_DIR = paths.ICDATA_DIR
DATA_DIR = paths.DATA_DIR
ANA_DIR = paths.ANA_DIR


#### Working at GT ####

LOCATION: 	 GT
USER: 	 cchen641
ICDATA_DIR: 	 /storage/home/hhive1/cchen641/data/icecube/data/analyses/online_GRECO/candidate-002-p10
DATA_DIR: 	 /storage/home/hhive1/cchen641/data/icecube/data/greco_grb/data
ANA_DIR: 	 /storage/home/hhive1/cchen641/data/icecube/data/greco_grb/data/csky_output



In [3]:
### create arguments arrays ---> 14.txt
# 0: grb_name
# 1: batchNtrials
# 2: batchIndex
# 3: ncpu

# Check 1 out of 2

In [5]:
df = pd.read_pickle(DATA_DIR+"/grbwebgbm/grbweb_gbm_noHealpix_2268.pkl")

In [11]:
####### Check before running ########
script_path    = '/scratch/cjchen/submit/14.sub'
## [::-1] reverse order
step = 6
grb_idx_starts = np.r_[0:2268:step]
grb_idx_ends = np.r_[step:2268+step:step]
batchNtrials = 1000
batchIndex = 0
ncpu = 8
#######################################

In [18]:
contents = ""
for grb_idx_start,grb_idx_end in zip(grb_idx_starts,grb_idx_ends):
    contents += "{}\t".format(grb_idx_start)
    contents += "{}\t".format(grb_idx_end)
    contents += "{}\t".format(batchNtrials)
    contents += "{}\t".format(batchIndex)
    contents += "{}\n".format(ncpu)

with open("14.txt", 'w') as writer:
    writer.write(contents)
    

In [19]:
print(contents)

0	6	1000	0	8
6	12	1000	0	8
12	18	1000	0	8
18	24	1000	0	8
24	30	1000	0	8
30	36	1000	0	8
36	42	1000	0	8
42	48	1000	0	8
48	54	1000	0	8
54	60	1000	0	8
60	66	1000	0	8
66	72	1000	0	8
72	78	1000	0	8
78	84	1000	0	8
84	90	1000	0	8
90	96	1000	0	8
96	102	1000	0	8
102	108	1000	0	8
108	114	1000	0	8
114	120	1000	0	8
120	126	1000	0	8
126	132	1000	0	8
132	138	1000	0	8
138	144	1000	0	8
144	150	1000	0	8
150	156	1000	0	8
156	162	1000	0	8
162	168	1000	0	8
168	174	1000	0	8
174	180	1000	0	8
180	186	1000	0	8
186	192	1000	0	8
192	198	1000	0	8
198	204	1000	0	8
204	210	1000	0	8
210	216	1000	0	8
216	222	1000	0	8
222	228	1000	0	8
228	234	1000	0	8
234	240	1000	0	8
240	246	1000	0	8
246	252	1000	0	8
252	258	1000	0	8
258	264	1000	0	8
264	270	1000	0	8
270	276	1000	0	8
276	282	1000	0	8
282	288	1000	0	8
288	294	1000	0	8
294	300	1000	0	8
300	306	1000	0	8
306	312	1000	0	8
312	318	1000	0	8
318	324	1000	0	8
324	330	1000	0	8
330	336	1000	0	8
336	342	1000	0	8
342	348	1000	0	8
348	354	1000	0	8
354	360	1000	0	8
360	366	1000	0	8

In [20]:
print("Total number of jobs: {}\n".format(contents.count('\n')))

Total number of jobs: 378



In [21]:
### create pbs file ---> 14.pbs

# Check 2 out of 2

In [22]:
nameofjob  = "14"
ntotal     = 378           # copy from the above 2nd cell "Total number of jobs: xxx"
inputtxt   = "14.txt"      # copy from the above 3rd cell
outdir     = ensure_dir("/storage/home/hhive1/cchen641/scratch/icecube/job_pbs_out/out")
walltimehr = 110 # 2hr is needed; 8 hrs for safe  # in Hour

In [30]:
contents = ""
contents += "#PBS -N {}\n".format(nameofjob)
############ update ncpu below ############
contents += "SBATCH --account=hive-itaboada3\n"
contents += "#PBS -l nodes=1:ppn={}\n".format(ncpu)           # number of nodes and cores per node required
contents += "#PBS -l pmem=4gb\n"                             # memory per core
contents += "#PBS -l walltime={}:00:00\n".format(walltimehr)  # dd:hh:mm:ss
contents += "#PBS -q hive\n"
contents += "#PBS -j oe\n"
contents += "#PBS -o {}\n".format(outdir+"/{}.out".format(nameofjob))
contents += "#PBS -t 1-{}%400".format(ntotal)
contents += "\n"
contents += "cd /storage/home/hhive1/cchen641/icecube/greco_grb/scripts\n"
contents += "icpy3\n" 
# Get Paramters from input.txt file using $PBS_ARRAYID as the line number 
contents += 'params=`sed -n "${{PBS_ARRAYID}} p" {}`\n'.format("hive_sub/"+inputtxt)    # outer single quote to preserve inner double quotes
contents += "paramsArray=($params)\n"
############ update args below ############
contents += "arg0=${paramsArray[0]}\n"
contents += "arg1=${paramsArray[1]}\n"
contents += "arg2=${paramsArray[2]}\n"
contents += "arg3=${paramsArray[3]}\n"
contents += "arg4=${paramsArray[4]}\n"
############ update script below ############
contents += "python 14_stack_TS_background.py "
############ update arg calls below ############
contents += "--grb_idx_start $arg0 --grb_idx_end $arg1 --batchNtrials $arg2 --batchIndex $arg3 --ncpu $arg4 "
print(contents)
############ update pbs name below ############
with open("14.pbs", 'w') as writer:
    writer.write(contents)

#PBS -N 14
SBATCH --account=hive-cchen641
#PBS -l nodes=1:ppn=8
#PBS -l pmem=4gb
#PBS -l walltime=110:00:00
#PBS -q hive
#PBS -j oe
#PBS -o /storage/home/hhive1/cchen641/scratch/icecube/job_pbs_out/out/14.out
#PBS -t 1-378%400
cd /storage/home/hhive1/cchen641/icecube/greco_grb/scripts
icpy3
params=`sed -n "${PBS_ARRAYID} p" hive_sub/14.txt`
paramsArray=($params)
arg0=${paramsArray[0]}
arg1=${paramsArray[1]}
arg2=${paramsArray[2]}
arg3=${paramsArray[3]}
arg4=${paramsArray[4]}
python 14_stack_TS_background.py --grb_idx_start $arg0 --grb_idx_end $arg1 --batchNtrials $arg2 --batchIndex $arg3 --ncpu $arg4 


In [38]:
contents = "#!/bin/bash"
contents += "#SBATCH -Jjobname={}\n".format(nameofjob)
############ update ncpu below ############
contents += "#SBATCH --account=hive-itaboada3\n"
contents += "#SBATCH -N 1 --ntasks-per-node={}\n".format(ncpu)           # number of nodes and cores per node required
contents += "#SBATCH --mem-per-cpu=4G\n"                             # memory per core
contents += "#SBATCH --time={}:00:00\n".format(walltimehr)  # dd:hh:mm:ss
contents += "#SBATCH -phive\n"
contents += "#SBATCH -oReport-%j.out\n"
contents += "#SBATCH --array=1-{}%400".format(ntotal)
contents += "\n"
contents += "cd /storage/home/hhive1/cchen641/icecube/greco_grb/scripts\n"
contents += "icpy3\n" 
# Get Paramters from input.txt file using $PBS_ARRAYID as the line number 
contents += 'params=`sed -n "${{PBS_ARRAYID}} p" {}`\n'.format("hive_sub/"+inputtxt)    # outer single quote to preserve inner double quotes
contents += "paramsArray=($params)\n"
############ update args below ############
contents += "arg0=${paramsArray[0]}\n"
contents += "arg1=${paramsArray[1]}\n"
contents += "arg2=${paramsArray[2]}\n"
contents += "arg3=${paramsArray[3]}\n"
contents += "arg4=${paramsArray[4]}\n"
############ update script below ############
contents += "srun python 14_stack_TS_background.py "
############ update arg calls below ############
contents += "--grb_idx_start $arg0 --grb_idx_end $arg1 --batchNtrials $arg2 --batchIndex $arg3 --ncpu $arg4 "
print(contents)
############ update pbs name below ############
with open("14.sbatch", 'w') as writer:
    writer.write(contents)

#!/bin/bash#SBATCH -Jjobname=14
#SBATCH --account=hive-itaboada3
#SBATCH -N 1 --ntasks-per-node=8
#SBATCH --mem-per-cpu=4G
#SBATCH --time=110:00:00
#SBATCH -phive
#SBATCH -oReport-%j.out
#SBATCH --array=1-378%400
cd /storage/home/hhive1/cchen641/icecube/greco_grb/scripts
icpy3
params=`sed -n "${PBS_ARRAYID} p" hive_sub/14.txt`
paramsArray=($params)
arg0=${paramsArray[0]}
arg1=${paramsArray[1]}
arg2=${paramsArray[2]}
arg3=${paramsArray[3]}
arg4=${paramsArray[4]}
srun python 14_stack_TS_background.py --grb_idx_start $arg0 --grb_idx_end $arg1 --batchNtrials $arg2 --batchIndex $arg3 --ncpu $arg4 
